In [1]:
import nltk
import numpy as np
import random
import json
import pickle
import pandas as pd
from tensorflow.keras.models import load_model
intents=json.loads(open("test_intents.json").read())
words=pickle.load(open ('words.pkl','rb'))
classes = pickle.load(open("classes.pkl","rb")) 
model=load_model("chatbot_model.h5")
import regex as re
from datetime import datetime

In [ ]:
# Assuming lemmatizer, words, model, and intents are pre-defined
lemmatizer = nltk.WordNetLemmatizer()

# This will store the current state of the conversation
conversation_state = {"stage": "start"}

def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

def bag_of_words(sentence):
    sentence_words = clean_up_sentence(sentence)
    bag = [0] * len(words)
    for w in sentence_words:
        for i, word in enumerate(words):
            if word == w:
                bag[i] = 1
    return np.array(bag)

def predict_class(sentence):
    bag_words = bag_of_words(sentence)
    res = model.predict(np.array([bag_words]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({'intent': classes[r[0]], 'probability': str(r[1])})
    return return_list
    
def get_response(tag, intents_json):
    list_of_intents = intents_json['intents']
    for intent in list_of_intents:
        if intent['tag'] == tag:
            return random.choice(intent['responses'])

def extract_number(user_input):
    # Extract the number of tickets using regex
    match = re.search(r'\b\d+\b', user_input)
    if match:
        return int(match.group())
    return None

def extract_date(user_input):
    # Extract date in dd/mm/yyyy format using regex
    match = re.search(r'\b\d{2}/\d{2}/\d{4}\b', user_input)
    if match:
        date_str = match.group()
        try:
            # Validate date format
            datetime.strptime(date_str, '%d/%m/%Y')
            return date_str
        except ValueError:
            return None
    return None
    

import random

def generate_ticket_number(museum, date):
    # Extract day part from the date string assuming format "YYYY-MM-DD"
    day = date.split("/")[0]  # Adjust based on your date format

    # Check for uniqueness of ticket number
    def generate_unique_number():
        random_number = random.randint(1000, 9999)
        ticket_number = f"{museum}/{day}/{random_number}"
        # Check if this ticket number already exists
        existing_ticket_numbers = [ticket["ticket_number"] for ticket in conversation_state["tickets"]]
        if ticket_number in existing_ticket_numbers:
            # Generate a new ticket number if it already exists
            return generate_unique_number()
        return ticket_number

    return generate_unique_number()


def handle_flow_based_responses(message, intents_json):
    global conversation_state

    # Predict the intent based on the user's message
    intents_list = predict_class(message)
    if not intents_list:
        return "Sorry, I didn't understand that."

    # Determine the recognized intent's tag
    intent_tag = intents_list[0]['intent']

    # Initialize tickets array if not already present
    if "tickets" not in conversation_state:
        conversation_state["tickets"] = []

    # Check current stage and handle response based on intent tag
    if conversation_state["stage"] == "start":
        if intent_tag == "start":
            conversation_state["stage"] = "manual_locate"
            return get_response("start", intents_json)
        elif intent_tag == "show_tickets":
            if not conversation_state["tickets"]:
                return "No tickets have been booked yet."
            else:
                tickets_info = "\n\n".join([
                    f"Ticket Number: {ticket['ticket_number']}\n"
                    f"Location: {ticket['location']}\n"
                    f"Museum: {ticket['museum']}\n"
                    f"Date: {ticket['date']}\n"
                    + "\n".join([
                        f"Person {i + 1}: Name: {person['name']}, Age: {person['age']}, Gender: {person['gender']}, Nationality: {person['nationality']}"
                        for i, person in enumerate(ticket["visitors"])
                    ])
                    for ticket in conversation_state["tickets"]
                ])
                return "Booked Tickets:\n" + tickets_info

    elif conversation_state["stage"] == "manual_locate":
        if intent_tag == "manual_locate":
            conversation_state["stage"] = "location_and_museum"
            return get_response("manual_locate", intents_json)
        elif intent_tag == "delete_ticket":
            if not conversation_state["tickets"]:
                return "No tickets have been booked yet."
            else:
                conversation_state["stage"] = "ask_ticket_number"
                return get_response("delete_ticket", intents_json)
        elif intent_tag == "show_tickets":
            if not conversation_state["tickets"]:
                return "No tickets have been booked yet."
            else:
                tickets_info = "\n\n".join([
                    f"Ticket Number: {ticket['ticket_number']}\n"
                    f"Location: {ticket['location']}\n"
                    f"Museum: {ticket['museum']}\n"
                    f"Date: {ticket['date']}\n"
                    + "\n".join([
                        f"Person {i + 1}: Name: {person['name']}, Age: {person['age']}, Gender: {person['gender']}, Nationality: {person['nationality']}"
                        for i, person in enumerate(ticket["visitors"])
                    ])
                    for ticket in conversation_state["tickets"]
                ])
                return "Booked Tickets:\n" + tickets_info
        else:
            return get_response("start", intents_json)

    # Handling the delete ticket state
    elif conversation_state["stage"] == "ask_ticket_number":
        ticket_number = message.strip()  # assuming the message contains the ticket number
        ticket_to_delete = next((ticket for ticket in conversation_state["tickets"] if ticket["ticket_number"] == ticket_number), None)
        if ticket_to_delete:
            conversation_state["tickets"].remove(ticket_to_delete)
            conversation_state["stage"] = "start"
            return get_response("ticket_deleted", intents_json).format(ticket_number=ticket_number)
        else:
            return get_response("invalid_ticket_number", intents_json)

    elif conversation_state["stage"] == "location_and_museum":
        parts = message.split(", ")
        if len(parts) == 2:
            location, museum_name = parts
            conversation_state["location_name"] = location
            conversation_state["museum_name"] = museum_name
            conversation_state["stage"] = "options"
            return get_response("location_and_museum", intents_json)
        else:
            return get_response("invalid_location_format", intents_json)

    elif conversation_state["stage"] == "options":
        if intent_tag == "book_ticket":
            conversation_state["stage"] = "ask_date"
            return get_response("book_ticket", intents_json)
        elif intent_tag == "about_museum":
            return get_response("about_museum", intents_json)
        elif intent_tag == "museum_timings":
            return get_response("museum_timings", intents_json)
        else:
            return get_response("invalid_option", intents_json)

    elif conversation_state["stage"] == "ask_date":
        date = extract_date(message)
        if date:
            conversation_state["date"] = date
            conversation_state["stage"] = "ask_tickets"
            return get_response("provide_date", intents_json).format(date=date)
        else:
            return get_response("invalid_date_format", intents_json)

    elif conversation_state["stage"] == "ask_tickets":
        number_of_tickets = extract_number(message)
        if number_of_tickets is not None and number_of_tickets > 0:
            conversation_state["tickets_count"] = number_of_tickets
            conversation_state["stage"] = "ask_details"
            return get_response("provide_tickets", intents_json).format(tickets=number_of_tickets)
        else:

            
            return get_response("invalid_tickets_number", intents_json)

    elif conversation_state["stage"] == "ask_details":
        if "people_details" not in conversation_state:
            conversation_state["people_details"] = []
        person_details = message.split(", ")
        if len(person_details) == 4:
            conversation_state["people_details"].append({
                "name": person_details[0],
                "age": person_details[1],
                "gender": person_details[2],
                "nationality": person_details[3]
            })
            if len(conversation_state["people_details"]) < conversation_state["tickets_count"]:
                return get_response("next_person_details", intents_json).format(count=len(conversation_state["people_details"]) + 1, total=conversation_state["tickets_count"])
            else:
                # Generate ticket number using the location and date, ensuring uniqueness
                ticket_number = generate_ticket_number(conversation_state["museum_name"], conversation_state["date"])
                ticket_info = {
                    "ticket_number": ticket_number,
                    "location": conversation_state["location_name"],
                    "museum": conversation_state["museum_name"],
                    "date": conversation_state["date"],
                    "visitors": conversation_state["people_details"]
                }
                conversation_state["tickets"].append(ticket_info)

                # Reset people details for the next set of bookings
                conversation_state["people_details"] = []

                # Receipt generation
                receipt = (
                    f"Booking Details:\n"
                    f"Ticket Number: {ticket_number}\n"
                    f"Location: {ticket_info['location']}\n"
                    f"Museum: {ticket_info['museum']}\n"
                    f"Date: {ticket_info['date']}\n"
                    + "\n".join([
                        f"Person {i + 1}: Name: {person['name']}, Age: {person['age']}, Gender: {person['gender']}, Nationality(Indian?): {person['nationality']}"
                        for i, person in enumerate(ticket_info["visitors"])
                    ])
                )
                conversation_state["stage"] = "start"
                return receipt + get_response("booking_complete", intents_json)
        else:
            return get_response("invalid_details_format", intents_json)

    conversation_state["stage"] = "start"
    return get_response("restart", intents_json)

# Example usage
while True:
    message = input("")
    response = handle_flow_based_responses(message, intents)
    print(response)


 start


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 163ms/step
Welcome! Please choose an option: a) Manual Locate b) Show Tickets c) Delete Tickets


 manual


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
Please enter your location and the museum name.


 kol, vict


...

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
Ticket number vict/12/2518 has been successfully deleted.


 start


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
Welcome! Please choose an option: a) Manual Locate b) Show Tickets c) Delete Tickets


 show tickets


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
No tickets have been booked yet.
